In [1]:
from google.colab import drive

drive.mount("/content/drive")

%cd /content/drive/MyDrive/nyu-stuff/2023-fall/DS-UA\ 301\ Advanced\ DS/final_project

Mounted at /content/drive
/content/drive/MyDrive/nyu-stuff/2023-fall/DS-UA 301 Advanced DS/final_project


In [2]:
%%capture
%pip install -r requirements.txt

In [1]:
import pandas
import numpy
import json5
import functools
import json
from matplotlib import pyplot
from typing import Optional, Any, Coroutine, Union, Dict, List
import nltk
from nltk.corpus import stopwords
import string
import contractions
import torch

nltk.download("averaged_perceptron_tagger")
nltk.download("universal_tagset")
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 8] nodename nor servname provided, or not
[nltk_data]     known>
[nltk_data] Error loading universal_tagset: <urlopen error [Errno 8]
[nltk_data]     nodename nor servname provided, or not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


False

In [4]:
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification", model="facebook/bart-large-mnli", device_map="auto"
)

In [5]:
sequence_to_classify = "one fine day I will see the world"
candidate_labels = ["travel", "cooking", "dancing"]
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'one fine day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.9946518540382385, 0.003005197737365961, 0.0023429866414517164]}

In [2]:
df_comments = pandas.read_csv("data/balanced_data.csv")

In [8]:
classes = ["good", "decent", "mediocre", "bad", "horrible"]

review_texts = df_comments["review_text"].tolist()

label_rows = []
n_reviews = len(review_texts)

from itertools import islice


def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


sz_batch = 512

batches = list(chunk(review_texts, sz_batch))
n_batches = len(batches)

for idx, batch in enumerate(batches):
    print(f"\rPROGRESS: {idx + 1} / {n_batches}", end="")

    results = classifier(list(batch), classes)

    for r in results:
        row = [r["sequence"]]
        for c in classes:
            row.append(r["scores"][r["labels"].index(c)])

        label_rows.append(row)

df_labels = pandas.DataFrame(label_rows, columns=["review_text"] + classes)
df_labels.to_csv("data/labels.csv", index=False)

print("\nAll Done.")

PROGRESS: 8 / 98

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


PROGRESS: 98 / 98
All Done.


In [28]:
df_merged = pandas.concat(
    [df_comments, df_labels.drop(columns=["review_text"])], axis=1
)

In [29]:
df_merged.to_csv("data/balanced_with_labels.csv", index=False)